# Conectando no google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalando dependencias

In [2]:
!pip install pandas
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-utnwnnfk/unsloth_466b0d850111478f8b1a254333fea36c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-utnwnnfk/unsloth_466b0d850111478f8b1a254333fea36c
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 11.5 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048 # delimita a quantidade de token que o modelo consegue olhar
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Download do modelo

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    # model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

# Prompt com o modelo não tunado

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
## Teste como modelo original sem o fine tunning
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Experiments with Soap (True Books: Science Experiments)", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Carregando o modelo com parametros

In [ ]:
perf_model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # podemos mudar aqui pois pode mudar algo
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # altera a intensidade da adaptação do lora
    lora_dropout = 0,
    bias = "none", # consideramos que nossos dados n etão enviesados

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Formatando a entrada de dados e carregando dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token # faz o modelo compreender o fim da sentença
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset


dataset = load_dataset("json", data_files='/content/drive/MyDrive/Colab Notebooks/trn_output.json', split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# Configurando treino do modelo

In [ ]:
trainer = SFTTrainer(
    model = perf_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 5,
        gradient_accumulation_steps = 8,
        warmup_steps = 8,
        max_steps = 80,
        learning_rate = 2e-5, # valor baixo para ele ficar atento a novos dados
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # otimiza o treinamento para usar menos memoria
        weight_decay = 0.15,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to=[]
    ),
)

# Treinando o modelo

In [ ]:
trainer_stats = trainer.train()

# Validando dados com o modelo treinado

In [ ]:
FastLanguageModel.for_inference(perf_model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Beautiful Assassin: A Novel", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = perf_model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
FastLanguageModel.for_inference(perf_model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Experiments with Soap (True Books: Science Experiments)", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Salvando o novo modelo treinado

In [ ]:
perf_model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-junior-2") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-junior-2") # saving tokenizer

# Carregando modelo já treinado


## Modelo 4

In [6]:
from unsloth import FastLanguageModel

llama_fine_tuning, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-v4", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(llama_fine_tuning)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

# Testando modelo ja treinado

In [7]:
import random
import pandas as pd
from datasets import load_dataset

random.seed(42)
dataset = load_dataset("json", data_files='/content/drive/MyDrive/Colab Notebooks/trn_output.json')

df = pd.DataFrame(dataset['train'])

sampled_items = df.sample(n=30, random_state=42)
sampled_items

Generating train split: 0 examples [00:00, ? examples/s]

,instruction,input,output
41475,Please provide a detailed description of the f...,Economics After the Crisis: Objectives and Mea...,"""A well-researched and profound rethink of mac..."
49837,Please provide a detailed description of the f...,I Am a Promise,"""Every child is full of promise, potential, an..."
72420,Please provide a detailed description of the f...,"Trapped (League of Peoples, Bk. 6)",James Alan Gardner is a graduate of the Clario...
134398,Please provide a detailed description of the f...,Dancing In Cadillac Light,"Gr 5-8-It is 1968 and Jaynell Lambert, an 11-y..."
139560,Please provide a detailed description of the f...,The Man Who Found Time: James Hutton And The D...,In this engaging account of scientific discove...
106363,Please provide a detailed description of the f...,The Cambridge Illustrated Atlas of Warfare: Re...,"""The atlas's elegant design and color illustra..."
142629,Please provide a detailed description of the f...,Daily Life Strategies for Teens,Feeling like chaos rules your world? Ready to ...
125791,Please provide a detailed description of the f...,Surveying (10th Edition),Surveying is essentially the science of measur...
32661,Please provide a detailed description of the f...,Getting Rid of Gout,Bryan Emmerson is at University of Queensland.
27145,Please provide a detailed description of the f...,Over 40 &amp; You're Hired!: Secrets to Landin...,Ryan (60 Seconds &amp; You're Hired!) offers a...


In [24]:
import re

def extract_response(text):
    # Find the "Response:" marker
    response_marker = "Response:"
    start_index = text.find(response_marker)

    if start_index == -1:
        return "No response found"

    # Extract everything after "Response:"
    response_text = text[start_index + len(response_marker):].strip()

    # Remove XML-like tags like <Response><![CDATA[...]]></Response>
    response_text = re.sub(r"<.*?>", "", response_text)

    return response_text.strip()

In [18]:
def inference(product):
  alpaca_prompt = """Instruction: Please provide a detailed description of the following product, including its features and any relevant specifications.
    Input: {}
    Response:"""


  inputs = tokenizer([alpaca_prompt.format(product)], return_tensors = "pt").to("cuda")

  outputs = llama_fine_tuning.generate(
      **inputs,
      max_new_tokens = 512,
      use_cache = True,
      temperature=0.7,   # Controls randomness (higher values make responses more random)
      top_p=0.9,         # Nucleus sampling (higher values allow more token diversity)
      top_k=50,          # Limits token selection to the top-K most likely tokens
      repetition_penalty=1.2,  # Penalizes repeated words/phrases (values > 1 reduce repetition)
      do_sample=True,    # Enables sampling (if False, uses greedy decoding)
      num_return_sequences=1)
  result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  return result[0]




In [19]:
sampled_items['result_modelo_v4'] = sampled_items['input'].apply(inference)
sampled_items['result_modelo_v4'] = sampled_items['result_modelo_v4'].apply(extract_response)
sampled_items.head()

,instruction,input,output,result_modelo_v4
41475,Please provide a detailed description of the f...,Economics After the Crisis: Objectives and Mea...,"""A well-researched and profound rethink of mac...",Instruction: Please provide a detailed descrip...
49837,Please provide a detailed description of the f...,I Am a Promise,"""Every child is full of promise, potential, an...",Instruction: Please provide a detailed descrip...
72420,Please provide a detailed description of the f...,"Trapped (League of Peoples, Bk. 6)",James Alan Gardner is a graduate of the Clario...,Instruction: Please provide a detailed descrip...
134398,Please provide a detailed description of the f...,Dancing In Cadillac Light,"Gr 5-8-It is 1968 and Jaynell Lambert, an 11-y...",Instruction: Please provide a detailed descrip...
139560,Please provide a detailed description of the f...,The Man Who Found Time: James Hutton And The D...,In this engaging account of scientific discove...,Instruction: Please provide a detailed descrip...
106363,Please provide a detailed description of the f...,The Cambridge Illustrated Atlas of Warfare: Re...,"""The atlas's elegant design and color illustra...",Instruction: Please provide a detailed descrip...
142629,Please provide a detailed description of the f...,Daily Life Strategies for Teens,Feeling like chaos rules your world? Ready to ...,Instruction: Please provide a detailed descrip...
125791,Please provide a detailed description of the f...,Surveying (10th Edition),Surveying is essentially the science of measur...,Instruction: Please provide a detailed descrip...
32661,Please provide a detailed description of the f...,Getting Rid of Gout,Bryan Emmerson is at University of Queensland.,Instruction: Please provide a detailed descrip...
27145,Please provide a detailed description of the f...,Over 40 &amp; You're Hired!: Secrets to Landin...,Ryan (60 Seconds &amp; You're Hired!) offers a...,Instruction: Please provide a detailed descrip...


In [25]:
sampled_items['result_modelo_v4'] = sampled_items['result_modelo_v4'].apply(extract_response)

In [29]:
sampled_items.to_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v4.csv')

### Cruzando resultados

In [51]:
results_fine_v0 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v0.csv')
results_fine_v1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v1.csv')
results_fine_v2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v2.csv')
results_fine_v3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v3.csv')
results_fine_v4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/results_modelo_v4.csv')

In [54]:
# Lista com todos os DataFrames e os respectivos nomes de colunas

df_final = results_fine_v0[['product_id', 'input', 'output', 'result_fine_tuning_v0']]

dfs = [
    results_fine_v1[['product_id', 'result_fine_tuning_v1']],
    results_fine_v2[['product_id', 'result_fine_tuning_v2']],
    results_fine_v3[['product_id', 'result_fine_tuning_v3']],
    results_fine_v4[['product_id', 'result_fine_tuning_v4']]
]

for df in dfs:
    df_final = df_final.merge(df, on='product_id', how='outer')  # Usa 'outer' para evitar perda de dados


In [57]:
df_final.to_csv('/content/drive/MyDrive/Colab Notebooks/all_results.csv')

In [59]:
df_final

,product_id,input,output,result_fine_tuning_v0,result_fine_tuning_v1,result_fine_tuning_v2,result_fine_tuning_v3,result_fine_tuning_v4
0,4083,The Pigman's Legacy,Starred Review. Grade 8 Up&mdash;In celebratio...,The Pigman's Legacy is a fictional book writte...,"""The Pigman's Legacy"" is a powerful, poignant,...","A master storyteller, Kipling wrote many stori...","""A fascinating, well-written story that has th...",A classic story of rural poverty set in 1960s ...
1,27145,Over 40 &amp; You're Hired!: Secrets to Landin...,Ryan (60 Seconds &amp; You're Hired!) offers a...,Over 40 &amp; You're Hired!: Secrets to Landin...,&quot;This book is a must-read for anyone who ...,Over 40 &amp; You're Hired!: Secrets to Landin...,"""This book will be a must read for anyone over...",Over 40 &#038; You&#8217;re Hired!\nA comprehe...
2,31564,Applied Epidemiology: Theory to Practice,"""A distinguished all-American multi-author off...",Applied Epidemiology: Theory to Practice is a ...,"""This book provides a comprehensive overview o...","Applied Epidemiology: Theory to Practice, Seco...","""This is a comprehensive text that covers epid...",Applied epidemiologists are public health prof...
3,32661,Getting Rid of Gout,Bryan Emmerson is at University of Queensland.,Getting rid of gout is a process that involves...,"""Getting Rid of Gout"" is a book that provides ...","""This book is a must-read for anyone who suffe...","""This book is very well written and is very ea...",This book is an absolute must-read for anyone ...
4,41475,Economics After the Crisis: Objectives and Mea...,"""A well-researched and profound rethink of mac...",This book by Robbins is a collection of his le...,"""The world has changed since the financial cri...",Economics After the Crisis: Objectives and Mea...,"""This book is a wonderful example of what the ...",The book examines the role that economic scien...
5,45970,Is He Mr. Right?: Everything You Need to Know ...,&#8220;An extremely helpful and insightful gui...,Is He Mr. Right?: Everything You Need to Know ...,NaN,Is He Mr. Right? will help you avoid the heart...,"""An indispensable guide for anyone considering...","""Is he MR. RIGHT?"" is an engaging new book tha..."
6,49837,I Am a Promise,"""Every child is full of promise, potential, an...",I Am a Promise is a book that promises to help...,I Am a Promise: The Journey of a New African C...,The story of a boy who discovers that he can m...,"In 1971, when the first edition of I Am a Prom...",A moving story about one girl's struggle to ri...
7,60304,"Radiographic Imaging &amp; Exposure, 2e (Faube...",Designed to ensure that readers grasp the info...,Radiographic imaging is the process of creatin...,NaN,"Radiographic Imaging &amp; Exposure, 2e (Faube...","""The text is clear and concise and the illustr...",Radiography is an imaging modality that has be...
8,60457,Diagnostic Atlas of Melanocytic Pathology: Exp...,"""This beautifully illustrated atlas complement...",Diagnostic Atlas of Melanocytic Pathology: Exp...,NaN,Diagnostic Atlas of Melanocytic Pathology: Exp...,"""A comprehensive, high-quality atlas of melano...",The book is written by two pathologists with o...
9,72420,"Trapped (League of Peoples, Bk. 6)",James Alan Gardner is a graduate of the Clario...,"Trapped (League of Peoples, Bk. 6) is a scienc...","""The League of Peoples is a galactic alliance ...","""A gripping, complex, and thought-provoking st...",Trapped is the sixth volume of the League of P...,"""In this stunning new novel from Hugo Award-wi..."


# 📊 Comparação de Parâmetros entre Modelos  

## 🏋️ Parâmetros de Treinamento  

| Parâmetro                     | v1  | v2  | v3  | v4  |
|--------------------------------|-----|-----|-----|-----|
| `per_device_train_batch_size`  | 2   | 4   | 3   | 5   |
| `gradient_accumulation_steps`  | 4   | 2   | 5   | 8   |
| `warmup_steps`                 | 5   | 10  | 6   | 8   |
| `max_steps`                    | 60  | 100 | 70  | 80  |
| `learning_rate`                | 2e-4 | 5e-5 | 2e-3 | 2e-5 |
| `fp16`                         | ✅  | ✅  | ✅  | ✅  |
| `bf16`                         | ✅  | ✅  | ✅  | ✅  |
| `logging_steps`                | 1   | 1   | 1   | 1   |
| `optim`                        | adamw_8bit | adamw_8bit | adamw_8bit | adamw_8bit |
| `weight_decay`                 | 0.01 | 0.01 | 0.05 | 0.15 |
| `lr_scheduler_type`            | linear | linear | linear | linear |
| `seed`                         | 3407 | 3407 | 3407 | 3407 |
| `output_dir`                   | outputs | outputs | outputs | outputs |
| `report_to`                    | []  | []  | []  | []  |

---

## 🚀 Parâmetros de Geração  

| Parâmetro               | v1  | v2  | v3  | v4  |
|-------------------------|-----|-----|-----|-----|
| `max_new_tokens`        | 500 | 500 | 500 | 512 |
| `use_cache`             | ✅  | ✅  | ✅  | ✅  |
| `temperature`           | -   | -   | -   | 0.7 |
| `top_p`                | -   | -   | -   | 0.9 |
| `top_k`                | -   | -   | -   | 50  |
| `repetition_penalty`   | -   | -   | -   | 1.2 |
| `do_sample`            | -   | -   | -   | ✅  |
| `num_return_sequences` | -   | -   | -   | 1   |

---

📌 **Notas:**  
- O símbolo ✅ indica que o parâmetro está ativado.  
- Os parâmetros de geração estavam incompletos para `v1`, `v2` e `v3`, então foram deixados em branco (`-`).  
- Essa estrutura permite rápida visualização das diferenças entre versões.  

In [43]:



### Video

1. Contexto do problema -- matheus

2. Preparação do Dataset -- jr

3. Executando fine tunnings -- jr

4. Resultados e Conclusão -- pedro

5. Como melhorar? -- pedro